In [10]:
import cv2
import mediapipe as mp

class HandDetector():
    def __init__(self, mode = False, max_hands = 2, det_rel_value = 1, tra_rel_value = 0.5):
        '''
        Initialization
        Arguments:
            mode: determine if receives static picture, default to be False
            max_hands: maximum number of hands which can be detected, default to be 2
            det_rel_value: minimum detection reliability value，default to be 1
            tra_rel_value: minimum tracking reliability value，default to be 0.5
        '''
        self.mode = mode
        self.max_hands = max_hands
        self.det_rel_value = det_rel_value
        self.tra_rel_value = tra_rel_value
        self.hands = mp.solutions.hands.Hands(self.mode, self.max_hands, self.det_rel_value, self.tra_rel_value)

    def find_hands(self, image, is_diagram = True):
        '''
        Hand detection
        Arguments:
            image: video frame picture
            is_diagram: if correctly draw the diagram of hands
        Return: 
            image: processed video frame picture
        '''
        imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Process the picture, detect whether gesture detected, save the data in self.results
        self.results = self.hands.process(imgRGB)
        if is_diagram:
            if self.results.multi_hand_landmarks:
                for hand_pos in self.results.multi_hand_landmarks:
                    mp.solutions.drawing_utils.draw_landmarks(image, hand_pos, mp.solutions.hands.HAND_CONNECTIONS)
        return image

    def find_positions(self, image, serial_num_hand = 0):
        '''
        Get gesture information
        Arguments:
            image: Video frame picture
            serial_num_hand: Hand's serial number, default to be 1
        Return:
            hand_list:
            Gesture information list, which is composed of ind(hand's serial number),
            pos_x(x_axis position on the screen), pos_y(y_axis position on the screen)
        '''
        self.hand_list = []
        if self.results.multi_hand_landmarks:
            hand = self.results.multi_hand_landmarks[serial_num_hand]
            for ind, pos in enumerate(hand.landmark):
                height, width, length = image.shape
                pos_x, pos_y = int(pos.x * width), int(pos.y * height)
                self.hand_list.append([ind, pos_x, pos_y])
        return self.hand_list


In [11]:
# six pictures of hands, representing number 0-5
target_pic_list = [
    'source/0.png',
    'source/1.png',
    'source/2.png',
    'source/3.png',
    'source/4.png',
    'source/5.png'
]
image_list = []

for ind in target_pic_list:
    finger_ind = cv2.imread(ind)
    image_list.append(finger_ind)
    

In [12]:

capture = cv2.VideoCapture(0)
# Create a gesture recognition object
detector = HandDetector()


# Fingertip list
# representing the fingertips of thumb, index finger, middle finger, ring finger and little finger in order
fingertip_ind = [4, 8, 12, 16, 20]

while True:
    is_detected, image = capture.read()

    if is_detected:
        # Detect gesture
        image = detector.find_hands(image, is_diagram = True)
        # Get gesture information
        finger_inf_list = detector.find_positions(image)
        if len(finger_inf_list) > 0:
            fingers = []
            for ind in fingertip_ind:
                # Find the position of each fingertip
                x, y = finger_inf_list[ind][1], finger_inf_list[ind][2]
                cv2.circle(image, (x, y), 10, (0, 255, 0), cv2.FILLED)
                # If thumb detected, 
                # check if the x position of the fingertip is greater than the x position of the second joint, 
                # if true, the thumb is considered to be open, 
                # otherwise, the thumb is considered to be closed.
                if ind == 4:
                    if finger_inf_list[ind][1] > finger_inf_list[ind - 1][1]:
                        fingers.append(1)
                    else:
                        fingers.append(0)
                # If other fingers detected,
                # check if the y position of the fingertip is greater than the y position of the second joint, 
                # if true, this finger is considered to be open, 
                # otherwise, this finger is considered to be closed.
                else:
                    if finger_inf_list[ind][2] < finger_inf_list[ind - 2][2]:
                        fingers.append(1)
                    else:
                        fingers.append(0)
            # fingers is a list containing 5 numbers of 0 or 1
            # 0 means one finger is closed, 1 means one finger is open
            finger_count = fingers.count(1)
            # Find the corresponding gesture picture and display it
            finger_img = image_list[finger_count]
            width, height, c = finger_img.shape
            image[0:width, 0:height] = finger_img
            cv2.rectangle(image, (200, 0), (300, 100), (0, 255, 0), cv2.FILLED)
            cv2.putText(image, str(finger_count), (200, 100), cv2.FONT_HERSHEY_DUPLEX, 5, (0, 0, 255))

        cv2.imshow('Finger Detection', image)

    k = cv2.waitKey(1)
    if k == ord('q'):
        break
